In [ ]:
import pandas as pd
import numpy as np
from pyteomics import fasta, mass, parser, cmass, auxiliary as aux
from scipy.stats import binom
from copy import copy
from collections import Counter
from os import path, listdir, environ
import matplotlib.pyplot as plt
import pickle
import random
from collections import defaultdict

In [ ]:
import sys; 
import ete3
from ete3 import NCBITaxa
ncbi = NCBITaxa()
import operator

In [ ]:
def calc_sf_all(v, n, p):
    sf_values = -np.log10(binom.sf(v-1, n, p))
    sf_values[np.isinf(sf_values)] = max(sf_values[~np.isinf(sf_values)])
    return sf_values

In [ ]:
# Set path to folders with sprot and uniprot databases generated in previous Notebook

path_to_fasta_dir_uniprot = '/home/kae-13-1/fasta/bacts_bases_uniprot/'
path_to_fasta_dir_sprot = '/home/kae-13-1/fasta/bacts_bases_sprot/'


In [ ]:
# Set path to files for preliminary search which should be generated only once
outfasta_path = '/home/fasta/prots_10_perc_23012024.fasta'
path_to_output_prot_set = '/home/fasta/prots_10_perc_peptides_mz_23012024.pickle'
path_to_output_specmap_id = '/home/fasta/specmapid_23012024.pickle'
path_to_output_cnt_to_spec = '/home/fasta/cnt_to_spec_23012024.pickle'

In [ ]:
# Set path to dictionaries generated in previous Notebook
with open('/home/kae-13-1/bact_Fast_search/dictionaries/species_leader_uniprot.pickle', 'rb') as f:
    species_leader_uniprot = pickle.load(f) # txid for species -> txid group leader by uniprot
with open('/home/kae-13-1/bact_Fast_search/dictionaries/species_leader_sprot.pickle', 'rb') as f:
    species_leader_sprot = pickle.load(f) # txid for species -> txid group leader by sprot
with open('/home/kae-13-1/bact_Fast_search/dictionaries/org_len_sprot.pickle', 'rb') as f:
    len_sprot = pickle.load(f) # txid -> the number of sprot proteins
with open('/home/kae-13-1/bact_Fast_search/dictionaries/org_len_uniprot.pickle', 'rb') as f:
    len_uniprot = pickle.load(f) #txid -> the number of uniprot proteins
with open('/home/kae-13-1/bact_Fast_search/dictionaries/exclude_names_set.pickle', 'rb') as f:
    exclude_names = pickle.load(f)

In [ ]:
# PREPARE 10% of DATABASE. Should be done only once

%%time
random_dict = {}
for k in len_uniprot.keys():
    random_dict[k] = 2000 / len_uniprot[k]
    
prots = []

outf = open(outfasta_path, 'w')
outf.close()

leaders = set(species_leader_sprot.values()).union(set(species_leader_uniprot.values()))
for f in leaders:
    prots = [] 
    if f in species_leader_sprot.values():
        n_prot = len_sprot[f]
    else: 
        n_prot = len_uniprot[f]
    if n_prot >= 200:
        for p in fasta.read(path.join(path_to_fasta_dir_uniprot, str(f) + '.fasta')):
            if p[0][:2] == 'sp' or random_dict[f] >= random.random():
                prots.append(p)
        fasta.write(prots, output=open(outfasta_path, 'a')).close()

In [ ]:
# prepare protein set. Should be done only once

%%time

# Add fixed modifications here

aa_mass = copy(mass.std_aa_mass)
aa_mass['C'] += 57.021464
aa_mass

prot_sets = defaultdict(list)
spec_map_id = dict()
cnt_to_spec = list()
spec_map_id_max = 0
spec_map_id_max_decoy = 0

cnt = 0
for p in fasta.read(outfasta_path):   
    
    if 'DECOY_' not in p[0]:
        cnt += 1
        if cnt % 1000000 == 0:
            print(cnt)

        decoy_flag = p[0].startswith('DECOY_')

        spec = ('DECOY_' if decoy_flag else '') + p[0].split('OX=')[-1].split(' ')[0]
        if spec not in spec_map_id:
            if not decoy_flag: 
                spec_map_id_max += 1
                spec_map_id[spec] = spec_map_id_max
            else:
                spec_map_id_max_decoy -= 1
                spec_map_id[spec] = spec_map_id_max_decoy
        
        spec_id = spec_map_id[spec]
        cnt_to_spec.append(spec_id)
        
        peptides = parser.cleave(p[1], '[RK]', 0, min_length=9)
        mz_list = []

        dont_use_fast_valid = parser.fast_valid(p[1])
        for pep in peptides:
            plen = len(pep)        
            if plen <= 15:
                if dont_use_fast_valid or parser.fast_valid(pep):
                    mz_list.append(cmass.fast_mass(pep, aa_mass=aa_mass))
        for mz in set(mz_list):
            prot_sets[mz].append(cnt)
                
pickle.dump(prot_sets, open(path_to_output_prot_set, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
pickle.dump(spec_map_id, open(path_to_output_specmap_id, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
pickle.dump(cnt_to_spec, open(path_to_output_cnt_to_spec, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import gc
gc.collect()

# RESTART NOTEBOOK HERE IF prot_sets were just created

In [ ]:
mass_accuracy = 4.0 # (in ppm)
mz_for_mass_accuracy = 1000 # (approximate max mz value)
mz_step = mass_accuracy * 1e-6 * mz_for_mass_accuracy
mz_step

In [ ]:
with open(path_to_output_prot_set, 'rb') as handle:
    prot_sets = pickle.load(handle)
spec_map_id = pickle.load(open(path_to_output_specmap_id, 'rb'))
cnt_to_spec= pickle.load(open(path_to_output_cnt_to_spec, 'rb'))

In [ ]:
spec_map_id_reversed = dict()
for k, v in spec_map_id.items():
    spec_map_id_reversed[v] = k

In [ ]:
protsN = Counter()

accurate_mz_map = defaultdict(list)

for v, prots in prot_sets.items():
    v_int = int(v/mz_step)
    accurate_mz_map[v_int].append(v)
    protsN.update(prots)

In [ ]:
def get_matches(df1, custom_range_mass_accuracy, score_threshold = 4.0):
    prots_spc = Counter()
    md_ar1 = []
    id_ar1 = []

    nmasses = df1['massCalib'].values
    charges = df1['charge'].values
    nmasses_int = df1['massCalib_int'].values
    nmasses_int_dict = defaultdict(set)
    for idx, nm in enumerate(nmasses_int):
        nmasses_int_dict[nm].add(idx)
        nmasses_int_dict[nm-1].add(idx)
        nmasses_int_dict[nm+1].add(idx)

    mz_acc_checked = set()

    for mz_int in accurate_mz_map:
        if mz_int in nmasses_int_dict:
            for mz_acc in accurate_mz_map[mz_int]:
                if mz_acc not in mz_acc_checked:
                    mz_acc_checked.add(mz_acc)
                    for idx_nm in nmasses_int_dict[mz_int]:
                        nm_val = nmasses[idx_nm]
                        mass_diff_ppm = (mz_acc - nm_val) / mz_acc * 1e6
#                         if abs(mass_diff_ppm) <= mass_accuracy:
                        if custom_range_mass_accuracy[0] <= mass_diff_ppm <= custom_range_mass_accuracy[1]:
#                                 md_ar1.append(mass_diff_ppm)
                            prots_spc.update(prot_sets[mz_acc])
                            for prot in prot_sets[mz_acc]:
                                md_ar1.append(mass_diff_ppm)
                                id_ar1.append(prot)
                            break


    prefix = 'DECOY_'
    isdecoy = lambda x: x[0].startswith(prefix)
    isdecoy_key_str = lambda x: x.startswith(prefix)
    isdecoy_key = lambda x: x < 0
    escore = lambda x: -x[1]

    top100decoy_score = [prots_spc.get(dprot, 0) for dprot in protsN]
    top100decoy_N = [val for key, val in protsN.items()]
    p = np.mean(top100decoy_score) / np.mean(top100decoy_N)
    print('p=%s' % (np.mean(top100decoy_score) / np.mean(top100decoy_N)))

    names_arr = np.array(list(prots_spc.keys()))
    v_arr = np.array([prots_spc[k] for k in names_arr])
    n_arr = np.array([protsN[k] for k in names_arr])

    prots_spc2 = dict()
    all_pvals = calc_sf_all(v_arr, n_arr, p)
    for idx, k in enumerate(names_arr):
        prots_spc2[k] = all_pvals[idx]   


    cnt = Counter()
    for k, v in prots_spc2.items():
        if v >= score_threshold:
            sp_id = cnt_to_spec[k-1]
            cnt[sp_id] += 1


    top_5_k = set()
    for k, v in cnt.most_common():
        if len(top_5_k) < 5:
            top_5_k.add(k)

            
    return cnt, top_5_k, md_ar1, id_ar1

In [ ]:
# Set path to folder with raw files
infolder = '/home/kae-13-1/bact_Zgoda_Mar2023/2024-02-28_uniprot_search_reduced/'

In [ ]:
# For the next step, you should install ThermoRawFileParser (https://github.com/compomics/ThermoRawFileParser)

In [ ]:
# Run ThermoRawFileParser for all raw files with an option to extract only MS1 spectra

for fn in os.listdir(infolder):
    if fn.endswith('.raw'):
        infile1 = os.path.join(infolder, fn)
        !ThermoRawFileParser -i $infile1 -L 1 -o $infolder

In [ ]:
# For the next step, you should install biosaur2 (https://github.com/markmipt/biosaur2)
!pip install biosaur2

In [ ]:
# Run biosaur2 for all mzML files to extract peptide isotope clusters.

for fn in os.listdir(infolder):
    if fn.endswith('.mzML'):
        mzmlname = os.path.join(infolder, fn)
        !biosaur2 $mzmlname -minlh 1

In [ ]:
%%time
n_files =  0
for z in listdir(infolder):
    if (z.endswith('features.tsv')):
        print(z)
        n_files+=1
#         break
        df1 = pd.read_table(path.join(infolder, z))
        df1 = df1[df1['nIsotopes'] >= 3]
        df1 = df1[df1['charge'] >= 2]
        df1 = df1[df1['charge'] <= 3]
        df1['massCalib_int'] = df1['massCalib'].apply(lambda x: int(x/mz_step))
        print(len(df1))
        
        cnt, top_5_k, md_ar1, id_ar1 = get_matches(df1, [-mass_accuracy, mass_accuracy], score_threshold=4.0)         
            
        md_ar2 = []
        for z1, z2 in zip(md_ar1, id_ar1):
            if cnt_to_spec[z2-1] in top_5_k:
                md_ar2.append(z1)

        from scipy.optimize import curve_fit

        def noisygaus(x, a, x0, sigma, b):
            return a * np.exp(-(x - x0) ** 2 / (2 * sigma ** 2)) + b


        md_ar2 = np.array(md_ar2)
        bbins = np.arange(min(md_ar2), max(md_ar2), 0.1)
        H2, b2 = np.histogram(md_ar2, bins=bbins)
        m, mi, s = max(H2), b2[np.argmax(H2)], (max(md_ar2) - min(md_ar2))/6
        noise = min(H2)
        popt, pcov = curve_fit(noisygaus, b2[1:], H2, p0=[m, mi, s, noise])
        shift, sigma = popt[1], abs(popt[2])
        print('Optimized mass shift and sigma: ', shift, sigma)

        custom_range_mass_accuracy = [shift-2*sigma, shift+2*sigma]
        
        cnt, _, md_ar1, id_ar1 = get_matches(df1, custom_range_mass_accuracy, score_threshold=4.0)    
            
            
        number_of_top_proteins = 15
    
        top_100_species_names = set()
        for k, v in cnt.most_common():
            if len(top_100_species_names) < number_of_top_proteins:
                k_orig = spec_map_id_reversed[k]
                if len_uniprot[int(k_orig)] < 220000:
                    if not int(k_orig) in exclude_names: ############################################################
                        top_100_species_names.add(k_orig) # OR k_orig???
                        orig_name = ncbi.get_taxid_translator([k_orig,])[int(k_orig)]
                        print(k, k_orig, orig_name, v)
            
        prots = []
        cnt = 0
        
        report = pd.DataFrame()
        
        for leader in top_100_species_names:
            SP = 0
            UN = 0
            
            if str(leader)+'.fasta' in listdir(path_to_fasta_dir_sprot):
                for p in fasta.read(path.join(path_to_fasta_dir_sprot, str(leader)+'.fasta')):
                    SP+=1
            for p in fasta.read(path.join(path_to_fasta_dir_uniprot, str(leader)+'.fasta')):
                prots.append(p)
                UN+=1
            report = pd.concat([report, pd.DataFrame.from_dict({'ID':[leader],
                                                               'Sprot':[SP],
                                                               'Uniprot':[UN]})])
            
        
        report.to_csv(path.join(infolder, z.replace('.features.tsv', '.strain_statistics.tsv')))
            
        random.shuffle(prots)        
        fasta.write(prots, output=open(path.join(infolder, z.replace('.features.tsv', '_top15_leaders_15.fasta')), 'w')).close()
        
        print('\n')

In [ ]:
# For the next step, you should install ms1searchpy (https://github.com/markmipt/ms1searchpy)
!pip install ms1searchpy

In [ ]:
# For the next step, you should install DeepLC (https://github.com/compomics/DeepLC)
# The recommended version is the clone available at https://github.com/markmipt/DeepLC
# The latest official DeepLC versions should work too, but ms1searchpy processing time will be much longer
!pip install https://github.com/markmipt/DeepLC/archive/refs/heads/alternative_best_model.zip

In [ ]:
# Run standard DirectMS1 analysis
for file in listdir(infolder):
    if (file.endswith('features.tsv')) and (file.replace('.tsv', '_proteins.tsv') not in listdir(infolder)):
        print(file)
        file_name = file.split('.features')[0]
        fasta_name = file_name + '_top15_leaders_15.fasta'
        fasta = path.join(infolder, fasta_name)
        file = path.join(infolder, file)
        !ms1searchpy $file -d $fasta -sc 3 -i 2 -nproc 10 -mc 0 -ad 1\
        -cmin 1 -ptol 8 -fdr 5 -ts 2 -ml 1\
        -deeplc 1